# <center>**Distance Measurement Process**<center>

#### **Goal:** obtain a distance error between the misclassified hand drawn spirals and the ideal model of an archimedean spiral.

Any functions / processes that require the use of tensorflow or keras need to be in this file. Other functions can be in a seperate python file. 

## **Process**
1. Preprocess the data for preparation to run through CNN model (need training, validation, and testing)
    - If model already trained, only the testing data is needed for evaluation
2. Evaluate the test data on the trained model 
    - Save and plot the results
3. Create an ideal archimedean spiral model 
    - This will be used for calculating distance, and eventually error, between the patient's hand drawn spiral and the mathematical model of the spiral
    - Plot the ideal spiral overlayed with the test spirals, along with their classifications
    - Need to get the spiral one-pixel-width thick

**Note: The following steps will ONLY be performed on spirals misclassified by the CNN**

4. Calculate the locations of the centers of the test spirals (using objectDetection.ipynb)
5.  Process each test spiral so that it is one-pixel-width thick
    - Consider erosion and skeletonization for this
6. Use the calculated locations of the center to unravel the spiral
    - Will also need to unravel the ideal spiral
7. Can then calculate the distance between the two and get an error measurement

*Instead of unravelling first, can also attempt to calculate the distance between the two spirals overlayed, but I think this is a more complicated approach if unravelling can be done*
